# Runtime (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common

In [ ]:
//// test

open FileSystem

## isWindows

In [ ]:
let inline isWindows () =
    System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform System.Runtime.InteropServices.OSPlatform.Windows

In [ ]:
//// test

isWindows ()

True

## splitCommand

In [ ]:
type private CommandParseStep =
    | Start
    | Path of quoted: bool
    | Arguments

let splitCommand (command: string) =
    let rec loop (path, args) chars step =
        match chars, step with
        | ('"' | '\'') :: tail, _ when path = "" -> loop (path, args) tail (Path true)
        | ('"' | '\'') :: tail, Path true -> loop (path, args) tail (Path false)
        | ' ' :: tail, Path true -> loop ($"{path} ", args) tail (Path true)
        | ' ' :: tail, (Start | Path _) -> loop (path, args) tail Arguments
        | char :: tail, Arguments -> loop (path, $"{args}{char}") tail Arguments
        | char :: tail, _ -> loop ($"{path}{char}", args) tail step
        | _, _ -> path |> String.replace @"\" "/", args
    let path, args = loop ("", "") (command |> Seq.toList) Start
    let workingDirectory, fileName =
        if path.StartsWith "./" || path.Contains "/"
        then path |> System.IO.Path.GetDirectoryName |> String.replace @"\" "/", System.IO.Path.GetFileName path
        else ".", path
    workingDirectory, fileName, args

In [ ]:
//// test

splitCommand ""
|> _equal (".", "", "")

splitCommand "/a/b/c"
|> _equal ("/a/b", "c", "")

splitCommand "cat file.txt"
|> _equal (".", "cat", "file.txt")

splitCommand @"..\..\file.exe file1.txt file2.txt"
|> _equal ("../..", "file.exe", "file1.txt file2.txt")

splitCommand @"c:\dir\file.exe ""file1.txt file2.txt"""
|> _equal (@"c:/dir", "file.exe", @"""file1.txt file2.txt""")

splitCommand @"""..\..\dir name\file.exe"" ""file 1.txt"" file2.txt"
|> _equal ("../../dir name", "file.exe", @"""file 1.txt"" file2.txt")

splitCommand @"""..\..\file 1.exe"" -c \\""echo 1\\"""
|> _equal ("../..", "file 1.exe", @"-c \\""echo 1\\""")

splitCommand @"..\..\file 1.exe -c \\""echo 1\\"""
|> _equal ("../..", "file", @"1.exe -c \\""echo 1\\""")

( ., ,  )
( /a/b, c,  )
( ., cat, file.txt )
( ../.., file.exe, file1.txt file2.txt )
( c:/dir, file.exe, "file1.txt file2.txt" )
( ../../dir name, file.exe, "file 1.txt" file2.txt )
( ../.., file 1.exe, -c \\"echo 1\\" )
( ../.., file, 1.exe -c \\"echo 1\\" )


## executeAsync

In [ ]:
let inline executeAsync (command : string) = async {
    let workingDirectory, fileName, arguments = command |> splitCommand
    let getLocals () = $"workingDirectory: {workingDirectory} / fileName: {fileName} / arguments: {arguments} / {getLocals ()}"
    
    let startInfo = System.Diagnostics.ProcessStartInfo (
        WorkingDirectory = workingDirectory,
        FileName = fileName,
        Arguments = arguments,
        RedirectStandardOutput = true,
        RedirectStandardError = true,
        UseShellExecute = false,
        CreateNoWindow = true
    )

    use proc = new System.Diagnostics.Process (StartInfo = startInfo)
    let result = System.Collections.Concurrent.ConcurrentStack<string> ()

    let inline event error (e: System.Diagnostics.DataReceivedEventArgs) =
        if e.Data <> null then
            trace
                (if error then Error else Debug)
                (fun () -> $"{if error then 'E' else ' '}{proc.Id}: {e.Data}")
                getLocals

            result.Push
                $"{
                    if error then '['.ToString() else System.String.Empty
                }{
                    e.Data
                }{
                    if error then ']'.ToString() else System.String.Empty
                }"

    proc.OutputDataReceived.Add (event false)
    proc.ErrorDataReceived.Add (event true)

    trace Debug (fun () -> $"executeAsync") getLocals

    if proc.Start () |> not
    then failwith $"executeAsync / proc.Start() error"

    proc.BeginErrorReadLine ()
    proc.BeginOutputReadLine ()

    do! proc.WaitForExitAsync () |> Async.AwaitTask

    let result = result |> Seq.rev |> String.concat System.Environment.NewLine

    trace Debug (fun () -> $"executeAsync / proc.ExitCode: {proc.ExitCode} / result.Length: {result.Length}") getLocals

    return proc.ExitCode, result
}

In [ ]:
//// test

let tempFolder = FileSystem.createTempDirectory ()
let path = tempFolder </> "test.txt"

let command = @$"pwsh -c ""Get-Content {path}"""

async {
    let! exitCode, result = executeAsync command
    exitCode |> _equal 1
    result |> _stringContains "not exist"

    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask

    let! exitCode, result = executeAsync command
    exitCode |> _equal 0
    result |> _equal "0"
}
|> Async.runWithTimeout 10000
|> _equal (Some ())

21:42:50 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-2142-5042-4270-4dacb1c87296 / result: { CreationTime = 2023-07-25 9:42:50 PM
  Exists = true }
21:42:50 #2 [Debug] executeAsync / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-2142-5042-4270-4dacb1c87296\test.txt"
21:42:51 #3 [Error] E22964: Get-Content: Cannot find path 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-2142-5042-4270-4dacb1c87296\test.txt' because it does not exist. / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-2142-5042-4270-4dacb1c87296\test.txt"
21:42:51 #4 [Debug] executeAsync / proc.ExitCode: 1 / result.Length: 171 / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230725-2142-5042-4270-4dacb1c87296\test.txt"
1
[Get-Content: Cannot 